## Classification of skin cancer

In [7]:
import wandb

# Only execute when training

# wandb.init(project="skin-cancer-detection")

## Loading the dataset

The dataset in processed for is too big to fit into memory *133 * 133 * 3 * float32*, even if we would use a data type with smaller precision. Our solution is to use tf.keras.utils.PyDataset as a base class for our dataset, and let it handle the dynamic loading of the data. The `create_dataset()` utility function uses this class to create a dataset object from the metadata that it receives.

In [ ]:
from preprocessing import create_dataset, load_metadata, upsample_metadata
from sklearn.model_selection import train_test_split

# Load the metadata and create train, test and validation split
metadata = load_metadata()
metadata = upsample_metadata(upsample_factor=5)
metadata_train, metadata_test = train_test_split(metadata, test_size=0.3)
metadata_test, metadata_valid = train_test_split(metadata_test, test_size=0.4)

# Load the dataset generators
batch_size = 5
ds_train = create_dataset(metadata_train, batch_size)
ds_test = create_dataset(metadata_test, batch_size)
ds_valid = create_dataset(metadata_valid, batch_size)

c:\BME\season_07\deep_learning\skin-cancer-detection\preprocessing.py:126: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(METADATA_PATH, dtype={"target": "int8"})


In [20]:
import pandas as pd
import numpy as np

df = pd.DataFrame({"age": np.arange(10)})
five = df[df["age"] == 5]

df = pd.concat([df, five])
df = pd.concat([df, five])
df

,age
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


### Class weights

Positive samples are heavily under-represented, which needs to be balanced out. We use the following techniques to compensate:

* `Sample weights`<br>
    For each sample the loss function is evaluated using a corresponding weight, <br>
    which is higher for the positive samples. We use to following formula: $c_d / (2 * c_s)$, <br>
    where $c_d$ is the count of all samples and $c_s$ is the count of samples for a given class of labels.

In [8]:
ds_train.class_weights

{0: 0.5004920400049917, 1: 508.588768115942}

In [9]:
# Construct the input shape from the size of the images
# and the number of channels (RGB)

input_shape = (*ds_train[0][0].shape[1:3], 3)
input_shape

(133, 133, 3)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Conv2D(16, kernel_size=3, activation="relu", input_shape=input_shape),
    # Conv2D(64, kernel_size=3, activation="relu"),
    Dropout(0.3),
    Flatten(),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

c:\BME\season_07\deep_learning\skin-cancer-detection\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 131, 131, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 131, 131, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 274576)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │     8,786,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,786,945 (33.52 MB)

 Trainable params: 8,786,945 (33.52 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
from tensorflow.keras.callbacks import EarlyStopping


model.fit(ds_train, batch_size=5, epochs=1, validation_data=ds_valid)

  351/56149 ━━━━━━━━━━━━━━━━━━━━ 43:16 47ms/step - accuracy: 0.9817 - loss: 0.0075

KeyboardInterrupt: 